## Segmentation and Generating Patches from each slide

### Segmentation tissues

Segmentation is operated at a best level for downsampling x64, which is generally at 2 or 3 level.

The values of `a_t` and `a_h` for segmentation should be set regrading:
- `a_t`: area filter threshold for tissue (positive integer, the minimum size of detected foreground contours to consider, relative to **a reference patch size of 512 x 512 at level 0**, e.g. a value 10 means only detected foreground contours of size greater than 10 512 x 512 sized patches at level 0 will be processed, default: 100)
- `a_h`: area filter threshold for holes (positive integer, the minimum size of detected holes/cavities in foreground contours to avoid, once again **relative to 512 x 512 sized patches at level 0**, default: 16)

When segmenting the tissues, the value of filter threshold would be automatically fixed in segmentation level.

When finished segmentation, all coordinates of foreground contours of tissues and holes were rescaled to level 0.

### Generating Patches

All patches are generated from the rectangle box bounding the foreground contours of tissues.

The most important parameters for patches are `patch_size`, `step_size` and `patch_level`, indicating which size at which level we want to extract. Actually when we run patching, the size and moving step is scaled to level 0 using following code:

```python
# Patch Size
patch_downsample = (int(self.level_downsamples[patch_level][0]), int(self.level_downsamples[patch_level][1]))
ref_patch_size = (patch_size * patch_downsample[0], patch_size * patch_downsample[1])
# Step Size
step_size_x = step_size * patch_downsample[0]
step_size_y = step_size * patch_downsample[1]
```
That is to say, all of generating patches are calculated at level 0. Naturally, the final coordinates restored in h5 files are at level 0. Also, the actual size of patch is equal to `patch_size * level_downsamples[patch_level]` at level 0. Of course, the patches are filtered if they are out of tissue. The checking function is `four_pt`: Easy version of 4pt contour checking function - 1 of 4 points need to be in the contour for test to pass.


As clarified above, **in order to get patches at the same resolution, we must make sure that `patch_size * level_downsamples[patch_level]` is always same for all slides**.

If the slides are from the same scanning instrument, **it's recommended to set `patch_level` as 1**.

### Runing code

Under CLAM package, run all above using the script:

```bash
cd ../scripts
nohup ./2-Segmentation-and-Generating-Patches.sh > 2-Seg-and-Patch.log 2>&1 &
```